# View postISRCCD from DM PostISRCCD 

- work with Weakly_2023_35

- author : Sylvie Dagoret-Campagne
- affiliation : IJCLab
- creation date : 2023/09/20
- last update : 2023/09/20


In [1]:
! eups list -s | grep LOCAL

analysis_tools        LOCAL:/home/d/dagoret/repos/repos_w_2023_35/analysis_tools 	setup
atmospec              LOCAL:/home/d/dagoret/repos/repos_w_2023_35/atmospec 	setup
eups                  LOCAL:/opt/lsst/software/stack/conda/miniconda3-py38_4.9.2/envs/lsst-scipipe-7.0.1/eups 	setup
spectractor           LOCAL:/home/d/dagoret/repos/repos_w_2023_35/Spectractor 	setup


In [2]:
! echo $IMAGE_DESCRIPTION
! eups list -s lsst_distrib

Weekly 2023_35
   g4213664e8e+c55b99ec95 	current w_2023_35 setup


In [3]:
import os
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.cm as cmx
import matplotlib.dates as mdates

import numpy as np
import pandas as pd
%matplotlib inline
from matplotlib.colors import LogNorm

from mpl_toolkits.axes_grid1 import make_axes_locatable

import matplotlib.ticker                         # here's where the formatter is
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)

from astropy.visualization import (MinMaxInterval, SqrtStretch,ZScaleInterval,PercentileInterval,
                                   ImageNormalize,imshow_norm)
from astropy.visualization.stretch import SinhStretch, LinearStretch,AsinhStretch,LogStretch


In [4]:
# LSST Display
import lsst.afw.display as afwDisplay
afw_display = afwDisplay.getDisplay(backend='firefly')

In [5]:
import lsst.daf.butler as dafButler
from lsst.daf.butler import CollectionType

In [6]:
#repo = "/sdf/group/rubin/repo/main" # repo/main
repo="/sdf/group/rubin/repo/oga/"  # /repo/embargo
butler = dafButler.Butler(repo)
registry = butler.registry

In [7]:
import warnings
warnings.filterwarnings("ignore")

# Registry

In [8]:
butler.registry.dimensions.getDatabaseElements()

NamedValueSet({DatabaseDimension(band), DatabaseDimension(detector), DatabaseDimension(physical_filter), DatabaseDimension(subfilter), DatabaseDimension(tract), DatabaseDimension(visit_system), DatabaseDimension(exposure), DatabaseDimension(patch), DatabaseDimension(visit), DatabaseDimensionCombination(visit_definition), DatabaseDimensionCombination(visit_detector_region), DatabaseDimensionCombination(visit_system_membership)})

In [9]:
print(butler.registry.dimensions["exposure"].RecordClass.fields)

exposure: 
  instrument: str
  id: int
  physical_filter: str
  obs_id: str
  exposure_time: float
  dark_time: float
  observation_type: str
  observation_reason: str
  day_obs: int
  seq_num: int
  seq_start: int
  seq_end: int
  group_name: str
  group_id: int
  target_name: str
  science_program: str
  tracking_ra: float
  tracking_dec: float
  sky_angle: float
  azimuth: float
  zenith_angle: float
  has_simulated: bool
  timespan: lsst.daf.butler.Timespan


In [10]:
print(butler.registry.dimensions["visit"].RecordClass.fields)

visit: 
  instrument: str
  id: int
  physical_filter: str
  name: str
  day_obs: int
  seq_num: int
  exposure_time: float
  target_name: str
  observation_reason: str
  science_program: str
  azimuth: float
  zenith_angle: float
  region: lsst.sphgeom.Region
  timespan: lsst.daf.butler.Timespan


In [11]:
print(butler.registry.dimensions["physical_filter"].RecordClass.fields)

physical_filter: 
  instrument: str
  name: str
  band: str


In [12]:
print(butler.registry.dimensions["tract"].RecordClass.fields)

tract: 
  skymap: str
  id: int
  region: lsst.sphgeom.Region


# Configuration

In [13]:
FLAG_PLOT=True

In [14]:
#DATE = 20230912
#FILTER="cyl_lens~holo4_001"
#FILTER="cyl_lens~holo4_003"

#my_collection = "u/dagoret/spectro/collimatorholo4003noflat/runs_20230912"
#FILTER="collimator~holo4_003"
#my_collection = "u/dagoret/spectro/emptyholo4003noflat/runs_20230912"
#FILTER="empty~holo4_003"
#my_collection = "u/dagoret/spectro/emptyholo4001noflat/runs_20230912"
#FILTER="empty~holo4_001"


#DATE = 20230913
#my_collection = "u/dagoret/spectro/collimatorholo4003noflat/runs_20230913"
#FILTER="collimator~holo4_003"
#my_collection = "u/dagoret/spectro/emptyholo4003noflat/runs_20230913"
#FILTER="empty~holo4_003"
#my_collection = "u/dagoret/spectro/emptyholo4001noflat/runs_20230913"
#FILTER="empt~holo4_001"



DATE = 20230914
my_collection = "u/dagoret/spectro/collimatorholo4003noflat/runs_20230914"
FILTER="collimator~holo4_003"
#my_collection = "u/dagoret/spectro/emptyholo4003noflat/runs_20230914"
#FILTER="empty~holo4_003"
#my_collection = "u/dagoret/spectro/emptyholo4001noflat/runs_20230914"
#FILTER="empty~holo4_001"


# read list of exposures

From the butler and the given user collection

In [15]:
datasetRefs = registry.queryDatasets(datasetType='postISRCCD', collections=my_collection, where= "instrument='LATISS'")

In [16]:
all_dataId = []
all_postisrccd  = []
all_exposures = []
for i, ref in enumerate(datasetRefs):
   
    print(f"========({i})================datasetType = postISRCCD ============================================")
    print("fullId..................:",ref.dataId.full)
    print("exposure................:",ref.dataId["exposure"])
    print("band....................:",ref.dataId["band"])
    print("physical filter.........:",ref.dataId["physical_filter"])
    print("run.....................:",ref.run)
    the_exposure = ref.dataId["exposure"]
    the_day_obs = ref.dataId["exposure"]//100_000
    the_seq_num = ref.dataId["exposure"]- the_day_obs*100_000    
    the_dataId = {'day_obs': the_day_obs,'seq_num':the_seq_num,'detector':0}
    print(the_dataId)
    #spec       = butler.get('spectraction',the_dataId)
    postisrccd = butler.get('postISRCCD', exposure=the_exposure, detector=0, collections=my_collection, instrument='LATISS')
    all_dataId.append(the_dataId) 
    all_exposures.append(the_exposure)
    all_postisrccd.append(postisrccd)
all_exposures = np.array(all_exposures)

========(0)================datasetType = postISRCCD ============================================
fullId..................: {band: 'white', instrument: 'LATISS', detector: 0, physical_filter: 'collimator~holo4_003', exposure: 2023091400120}
exposure................: 2023091400120
band....................: white
physical filter.........: collimator~holo4_003
run.....................: u/dagoret/spectro/collimatorholo4003noflat/runs_20230914/20230915T091136Z
{'day_obs': 20230914, 'seq_num': 120, 'detector': 0}
========(1)================datasetType = postISRCCD ============================================
fullId..................: {band: 'white', instrument: 'LATISS', detector: 0, physical_filter: 'collimator~holo4_003', exposure: 2023091400095}
exposure................: 2023091400095
band....................: white
physical filter.........: collimator~holo4_003
run.....................: u/dagoret/spectro/collimatorholo4003noflat/runs_20230914/20230915T091136Z
{'day_obs': 20230914, 'seq_nu

In [17]:
N = len(all_postisrccd)

In [18]:
all_exposures_sortedindexes = np.argsort(all_exposures)

In [19]:
all_exposures[all_exposures_sortedindexes]

array([2023091400094, 2023091400095, 2023091400108, 2023091400109,
       2023091400120, 2023091400121, 2023091400132, 2023091400133,
       2023091400145, 2023091400146, 2023091400157, 2023091400158,
       2023091400169, 2023091400170, 2023091400182, 2023091400183,
       2023091400194, 2023091400195, 2023091400215, 2023091400216,
       2023091400227, 2023091400228, 2023091400239, 2023091400240,
       2023091400252, 2023091400253])

In [20]:
#for idx in range(N):
#    idx_sorted = all_exposures_sortedindexes[idx]
#    dm_postisrccd = all_postisrccd[idx_sorted]
#    dm_postisrccd_md = dict(dm_postisrccd.getMetadata())
#    print(f"==================={all_exposures[idx_sorted]}=======================")
#    print('LSST CALIB DATE BIAS :',dm_postisrccd_md['LSST CALIB DATE BIAS'])
#    print('LSST CALIB UUID BIAS :',dm_postisrccd_md['LSST CALIB UUID BIAS']) 
#    print('LSST CALIB RUN BIAS  :',dm_postisrccd_md['LSST CALIB RUN BIAS'])
#    print("-------")
#    print('LSST CALIB DATE DEFECTS :',dm_postisrccd_md['LSST CALIB DATE DEFECTS'])
#    print('LSST CALIB UUID DEFECTS :',dm_postisrccd_md['LSST CALIB UUID DEFECTS']) 
#    print('LSST CALIB RUN DEFECTS  :',dm_postisrccd_md['LSST CALIB RUN DEFECTS'])

## Select flags options

In [21]:
FLAG_ROTATE_IMG = True
FLAG_TRANSFORM = True

## Transformations
astropy scale transformations

In [22]:
transform = AsinhStretch() + PercentileInterval(99.)
#transform = PercentileInterval(98.)

In [23]:
idx=10
idx_sorted = all_exposures_sortedindexes[idx]
dm_postisrccd = all_postisrccd[idx_sorted]
dm_postisrccd_md = dict(dm_postisrccd.getMetadata())
exposure_selected =all_exposures[idx_sorted]

print(exposure_selected)
raw_img= butler.get('raw', dataId={'exposure': exposure_selected, 'instrument': 'LATISS', 'detector': 0}, collections = ['LATISS/calib','LATISS/raw/all',] )
meta = raw_img.getMetadata()
md = meta.toDict()
      
the_object = md['OBJECT']
the_am= md['AMSTART']
the_filter=md['FILTER']

2023091400157


In [24]:
afw_display.scale('linear', 'zscale',None)
the_title = f"{idx} :: dm postisrccd exposure : {exposure_selected}, target={the_object}, airmass={the_am:.2f}, filter={the_filter}"
#le titre ne marche pas avec firefly
#afw_display.mtv(dm_postisrccd.image,title=the_title)
afw_display.mtv(dm_postisrccd.image)

In [25]:
print("finished")

finished
